このレポのファイルがみんな古すぎてあまりここに置きたくないが、適切な場所がここしか思いあたらない。


In [ ]:
from logging import getLogger, INFO, DEBUG
import numpy as np
import networkx as nx
from clustice.geometry import make_layout
from clustice import graph
from cycless import cycles
from icecream import ic


def volume(pos: np.ndarray, g: nx.Graph) -> float:
    """Volume of the given pseudo-polyhedron

    Args:
        pos (np.ndarray): positions of the nodes
        g (nx.Graph): graph representation of the polyhedron

    Returns:
        float: volume
    """
    # 擬多面体の重心
    volume_center = np.mean(pos, axis=0)
    # 擬多面体の体積は、四角錐の体積の総和
    total_volume = 0
    # すべての面について
    for cycle in cycles.cycles_iter(g, maxsize=8):
        # print(cycle)
        # 輪っかにする。
        cycle = list(cycle)
        cycle.append(cycle[0])
        # 頂点の座標
        face_pos = pos[list(cycle)]
        # 面の中心
        face_center = np.mean(face_pos, axis=0)
        face_volume = 0
        # それぞれの辺について
        for i, j in zip(cycle, cycle[1:]):
            # 面の中心と辺(i,j)と擬多面体の重心で四面体を張る
            tri_pos = np.array([pos[i], pos[j], face_center]) - volume_center
            # 行列式で四角錐の体積を求める。
            tetra_volume = np.linalg.det(tri_pos) / 6
            # print(np.abs(tetra_volume))
            face_volume += np.abs(tetra_volume)
        # print(face_volume)
        total_volume += face_volume
    return total_volume


def sph_angle(a: np.ndarray, b: np.ndarray, c: np.ndarray) -> float:
    """Area of a spherical triangle consisting of three spherical line segments.

    Args:
        a, b, c (np.ndarray): Lengths of edges on a unit sphere

    Returns:
        float: area of a spherical triangle
    """

    # Euler's expression https://ja.wikipedia.org/wiki/球面三角法
    ea = a / np.linalg.norm(a)
    eb = b / np.linalg.norm(b)
    ec = c / np.linalg.norm(c)
    cosa = eb @ ec
    cosb = ec @ ea
    cosc = ea @ eb

    cosah = ((1 + cosa) / 2) ** 0.5
    cosbh = ((1 + cosb) / 2) ** 0.5
    cosch = ((1 + cosc) / 2) ** 0.5

    return 2 * np.arccos((1 + cosa + cosb + cosc) / (4 * cosah * cosbh * cosch))


def inner_sph_angles(pos: np.ndarray, g: nx.Graph) -> float:
    # 擬多面体の重心
    volume_center = np.mean(pos, axis=0)

    centers = dict()
    for cycle in cycles.cycles_iter(g, maxsize=8):
        # 各辺が属する面の中心座標が必要になるので、記録しておく。
        # 頂点の座標
        face_pos = pos[list(cycle)]
        # 面の中心
        face_center = np.mean(face_pos, axis=0)

        for node in cycle:
            if node not in centers:
                centers[node] = []
            centers[node].append(face_center)
    # ic(centers)
    stereo = 0
    for node in g:
        deg = len(g[node])
        assert deg <= 4, "More than four edges at a node."
        if deg == 4:
            # 4結合は完全な球
            stereo += 4 * np.pi
        elif deg < 2:
            # 1結合は体積に寄与しない
            pass
        elif deg == 2:
            i, j = g[node]
            assert len(centers[node]) == 2, centers[node]
            for k in centers[node]:
                s = sph_angle(pos[i] - pos[node], pos[j] - pos[node], k - pos[node])
                # ic(s)
                stereo += s
        else:
            # deg == 3:
            # 頂点ノードの隣接3点に関して
            for i in g[node]:
                # それが属する面の中心について
                for k in centers[i]:
                    s = sph_angle(
                        pos[i] - pos[node], volume_center - pos[node], k - pos[node]
                    )
                    # ic(s)
                    stereo += s
    return stereo


logger = getLogger()
logger.setLevel(INFO)

# O-O distance
L = 0.276  # nm

# they are not vitrites.
# g = graph.great_icosahedron(8, separation=L)
# g = graph.great_decahedron(4)
# g = graph.twistane()
# g = nx.cycle_graph(6) # hexagon
# g = nx.cycle_graph(7) # heptagon


# g = graph.small_barrelan()
# g = graph.large_barrelan()
# g = graph.hex_ice()
g = graph.adamantane()
# g = nx.cubical_graph()  # cubic octamer
# g = nx.dodecahedral_graph()

# estimate of the positions of the nodes
mol_positions = make_layout(g, edge_length=L)

# サイズのチェック
# for i, j in g.edges():
#     ic(np.linalg.norm(mol_positions[i] - mol_positions[j]))

# 体積計算
vol = volume(mol_positions, g)
ic(vol)

# 原子数の計算
atoms = inner_sph_angles(mol_positions, g) / (4 * np.pi)
ic(atoms)

# 水の密度を計算してみよう
NA = 6.022e23
density = 18 * atoms / (vol * 1e-21 * NA)
ic(density)
# density = 18 * 8 / ((L * 4 / 3**0.5) ** 3 * 1e-21 * NA)
# ic(density, "cubic ice")